In [1]:
import sys
sys_path = 'D:/Research_CAE/MyTinyCUDANN/tiny-cuda-nn/main'
sys.path.append(sys_path)
from my_tiny_cuda import my_MLP
from tools import ShiftedLegendrePolynomial2D, generate_grid_points, sample_all_boundary
from gradient import grad1, grad2
import torch
import numpy as np
import tinycudann as tcnn
import json
device = 'cuda'

In [2]:
class Heat_equation(torch.nn.Module):
    def __init__(self,k = 1, L = 1):
        super().__init__()
        self.k = k
        self.L = L
        self.tau = 1e-4
    def strong_form(self,X,grad_result):
        du_dt,d2u_dx2 = grad_result["du_dx"],grad_result["d2u_dy2"]
        result = du_dt - self.k*d2u_dx2
        return result
    def weak_form(self,X,grad_result, grad_result_v):
        # The shape of grad_result is (batch_size)
        # The shape of grad_result_v is (batch_size,max_order^2)
        du_dt,du_dx = grad_result["du_dx"].unsqueeze(1),grad_result["du_dy"].unsqueeze(1)
        v, dv_dx = grad_result_v["u"],grad_result_v["du_dy"]
        result = du_dt* v + du_dx * dv_dx
        return result
    def BC_function(self,X):
        # return torch.sin(X[:,0]*torch.pi)*torch.sin(X[:,1]*torch.pi)
        return self.real_solution(X)
    def real_solution(self,X):
        return torch.exp(-self.k*(torch.pi/self.L)**2*X[:,0])*torch.sin(X[:,1]*torch.pi/self.L)

class real_solution(torch.nn.Module):
    def __init__(self,k = 1, L = 1):
        super().__init__()
        self.equation = Heat_equation(k,L)
    def forward(self,X):
        return self.equation.real_solution(X)

In [30]:
equation = Heat_equation()
grid_points_resolution = 300
base_grid_points = generate_grid_points(grid_points_resolution, field_min = 0, field_max = 1 - 1/grid_points_resolution)
model_real = real_solution()
batch_size = base_grid_points.shape[0]
diff_info = grad1(model_real, batch_size)
diff2_info = grad2(model_real, batch_size)
diff2_info.to_device(device)
grad_result_2 = diff2_info(base_grid_points)
du_dx,du_dy,u = diff_info.forward_2d(base_grid_points)
grad_result = {"du_dx":du_dx,"du_dy":du_dy,"u":u}
max_order = 10
test_function = ShiftedLegendrePolynomial2D(max_order).to(device)
v = test_function(base_grid_points[:,0],base_grid_points[:,1])
dv_dx, dv_dy = test_function.derivatives(base_grid_points[:,0],base_grid_points[:,1])
grad_result_v = {"u":v,"du_dx":dv_dx,"du_dy":dv_dy}
# dv_dy = torch.autograd.grad(v,base_grid_points,torch.ones_like(v),create_graph = True)
# grad_result_v = {"u":v,"du_dy":dv_dy}
# v.shape,dv_dy

In [15]:
PDE_strong_residual= equation.strong_form(base_grid_points,grad_result_2)
(PDE_strong_residual**2).mean()

tensor(3.2022e-14, device='cuda:0', grad_fn=<MeanBackward0>)

In [31]:
PDE_residual = equation.weak_form(base_grid_points,grad_result,grad_result_v)
(PDE_residual**2).mean()

tensor(13.8145, device='cuda:0', grad_fn=<MeanBackward0>)